In [1]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
df=pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
# splitting fromṁataset_sp

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [8]:
# split train and test

In [19]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [14]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

model.fit(X_train_scaled,y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 8ms/step - loss: 0.5437 - accuracy: 0.8182
Epoch 2502/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5437 - accuracy: 0.8182
Epoch 2503/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5437 - accuracy: 0.8182
Epoch 2504/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5437 - accuracy: 0.8182
Epoch 2505/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5436 - accuracy: 0.8182
Epoch 2506/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5436 - accuracy: 0.8182
Epoch 2507/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5436 - accuracy: 0.8182
Epoch 2508/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5435 - accuracy: 0.8182
Epoch 2509/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5435 - accuracy: 0.8182
Epoch 2510/5000
1/1 [=============================

In [20]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 152ms/step - loss: 1.0002 - accuracy: 0.5000


[1.0001877546310425, 0.5]

In [21]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 76ms/step


array([[0.15454699],
       [0.15246725],
       [0.07236988],
       [0.15318179],
       [0.15469128],
       [0.15555929]], dtype=float32)

In [22]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [23]:
coef,intercept=model.get_weights()

In [24]:
coef,intercept

(array([[0.05519083],
        [0.8398784 ]], dtype=float32), array([-2.5651925], dtype=float32))

In [26]:
def signmoid(x):
  import math
  return 1 / (1+math.exp(-x))

signmoid(18)

0.9999999847700205

In [27]:
X_test

,age,affordibility
2,47,1
10,18,1
21,26,0
11,28,1
14,49,1
9,61,1


In [29]:
def prediction_function(age,affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability+intercept
  return signmoid(weighted_sum)

prediction_function(.47,1)

0.15454698482439672

In [33]:
def prediction_function(age,affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability+intercept
  return signmoid(weighted_sum)
prediction_function(.18,1)

0.15246724324482636

In [34]:
# now we start implementing graient descent in python .again goal is to come up with same w1,w2 and bias that kkeras 

In [35]:
# write 1st couple of helper routines such as sigmois and log_loss

In [36]:
def sigmoid_numpy(X):
  return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [38]:
def log_loss(y_true,y_predicted):
  epsilon =1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [41]:
def gradient_descent(age,affordability,y_true,epochs,loss_threshold):
  w1=w2=1
  bias=0
  rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = w1+w2*affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true,y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)
    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias -rate * bias_d

    print(f'Epoch:{i},w1:{w1},w2:{w2},bias:{bias},loss:{loss}')

    if loss<=loss_threshold:
      break
  return w1,w2,bias
  
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4631)

Epoch:0,w1:0.9574035992193684,w2:0.9242918388252193,bias:-0.16317335728932708,loss:0.9219584388521546
Epoch:1,w1:0.922066545612089,w2:0.8590387201261663,bias:-0.30819718927091166,loss:0.8477453787991525
Epoch:2,w1:0.894043431257712,w2:0.8047435847248681,bias:-0.4350055880357564,loss:0.7908303619790257
Epoch:3,w1:0.8729932946214354,w2:0.7612101934232305,bias:-0.5445000522470408,loss:0.7488782034996778
Epoch:4,w1:0.8582730917516097,w2:0.7276519612576007,bias:-0.6383115052127857,loss:0.7189518878989642
Epoch:5,w1:0.8490705417082812,w2:0.7029044644239018,bias:-0.7184716833177047,loss:0.6980986173334993
Epoch:6,w1:0.8445276062982066,w2:0.6856469074565564,bias:-0.7871116743425546,loss:0.6837546473269219
Epoch:7,w1:0.8438279013299365,w2:0.6745716001676707,bias:-0.8462506254537895,loss:0.6739111561295054
Epoch:8,w1:0.8462447873733275,w2:0.6684857306370656,bias:-0.8976809166689936,loss:0.6671033331068315
Epoch:9,w1:0.8511593012562954,w2:0.6663570667317269,bias:-0.942926635482515,loss:0.66231430

(16.357013497954423, 1.2650806866450972, -17.034307520639576)

In [42]:
coef,intercept

(array([[0.05519083],
        [0.8398784 ]], dtype=float32), array([-2.5651925], dtype=float32))